# Data Cleaner + Phone Number Scraper -> Output to Discord Bot chat logs

This project is a personal project taken on for a friend in the Real Estate Space. 

# Imports

In [434]:
import numpy as np
import pandas as pd
import io
import os
import time
import undetected_chromedriver as uc
import requests
import bs4
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from openpyxl import load_workbook

In [435]:
# Read in raw data
raw_df = pd.read_csv('raw_housing_data.csv')

# Variables

In [ ]:
chromedriver_path = "C:\\Users\\kin\\Desktop\\chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)
url = "http://www.usnews.com/best-colleges/rankings/national-universities"
soup = BeautifulSoup(driver.page_source, 'html.parser')
discord = 'https://discord.com/api/webhooks/1169906230316240937/0EXUlGJRLPb1Tclm-cWNTeWmd-mCiNc7t80gtMTIY3a3DY-eGpje8BKxuu4m1TANQiNT'
discord2 = 'https://discord.com/api/webhooks/1169922010848174132/pE5O41sBx1Hqjbs3iis3aQdbwIJG57TFt6vv6NBNWtNk8lXdwA72o_4cHk1WgdwWD41P'
phones = []

# Functions

In [436]:
# Cleaning data, drop NA's, filter out specific houses, rename columns

def clean_data(df):
    df2 = raw_df[['OwnerNmFirst', 'OwnerNm2First', 'OwnerNmLast', 'SiteAddr']]
    df3 = df2[df2["OwnerNmLast"].str.contains("LLC" or "Corp" or "Trust") == False]
    df3 = df3[df3['OwnerNmFirst'].notna()]
    df3.columns = ['first', 'middle', 'last', 'addy']
    df4 = df3[df3["addy"].str.contains("#") == False]
    df_final = df4.reset_index(drop=True)
    
    return df_final

In [437]:
def split_addy(df):
    split_addys = []
    for i in df['addy']:
        splitString = i.split(' ')
        if splitString[-1].isnumeric():
            splitString.pop(-1)
        split_addys.append(splitString)
    split_addy_df = pd.DataFrame(split_addys, columns=['house_num','direction', 'street_name', 'street_name2', 'street_name3', 'street_type'])
    return split_addy_df

In [438]:
def create_addy_link(final_df):
    temp_list = []
    for i in final_df.index:
        link = final_df['house_num'][i] + "-" + final_df['direction'][i] + "-" + final_df['street_name'][i]
        if final_df['street_name2'][i] is not None:
            link = link + "-" + final_df['street_name2'][i]
            if final_df['street_name3'][i] is not None:
                link = link + "-" + final_df['street_name3'][i]
                if final_df['street_type'][i] is not None:
                    link = link + "-" + final_df['street_type'][i]
        final_link = "https://www.fastpeoplesearch.com/address/" + link + "_tucson-az"
        temp_list.append(final_link)
    new_df = pd.DataFrame(temp_list, columns=['addy_link'])
    return new_df

In [567]:
def find_phone_num(id):
    chromedriver_path = "C:\\Users\\kin\\Desktop\\chromedriver.exe"
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service)
    driver.get('https://www.fastpeoplesearch.com/' + id)
    # Sleep if denied
    if "Access Denied" in driver.page_source:
        print("Access Denied")
        time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    phone_num = soup.find('a', title=lambda x: x and "Search people associated with the phone number" in x)
    if phone_num:
        driver.quit()
        return phone_num.text
    else:
        driver.quit()
        return("no num found")

In [597]:
df = clean_data(raw_df)
df2 = split_addy(df)
df3 = create_addy_link(df2)
final_df = pd.concat([df, df2, df3], axis=1, join="inner")

In [587]:
count = 780
for index, value in enumerate(final_df['addy_link'], start=780): 
    print(list((index, value))) 
    print(count)
    current_name = final_df['first'][count] + '-' + final_df['last'][count]
    print('searching for ' + current_name)
    print("---------")
    count += 1

[780, 'https://www.fastpeoplesearch.com/address/5820-N-San-Joaquin-Av_tucson-az']
780
searching for John-Arredondo
---------
[781, 'https://www.fastpeoplesearch.com/address/2506-W-Ruthrauff-Rd_tucson-az']
781
searching for Jose-Munoz
---------
[782, 'https://www.fastpeoplesearch.com/address/4950-N-Maryvale-Av_tucson-az']
782
searching for Richard-Ortiz
---------
[783, 'https://www.fastpeoplesearch.com/address/2850-W-Diamond-St_tucson-az']
783
searching for Elizabeth-Newton
---------
[784, 'https://www.fastpeoplesearch.com/address/6219-N-Canyon-Dr_tucson-az']
784
searching for Christina-Fraijo
---------
[785, 'https://www.fastpeoplesearch.com/address/3275-N-Grannen-Rd_tucson-az']
785
searching for Ray-Rodriguez
---------
[786, 'https://www.fastpeoplesearch.com/address/5141-N-Kevy-Pl_tucson-az']
786
searching for Jesus-De La Rosa
---------
[787, 'https://www.fastpeoplesearch.com/address/1422-W-Rillito-St_tucson-az']
787
searching for Armando-Molina
---------
[788, 'https://www.fastpeople

KeyError: 1968

In [598]:
final_df = final_df.iloc[780:,:]
final_df.reset_index(inplace=True,drop=True)


In [599]:
final_df

,first,middle,last,addy,house_num,direction,street_name,street_name2,street_name3,street_type,addy_link
0,John,Elena,Arredondo,129 S Melrose Av,129,S,Melrose,Av,None,None,https://www.fastpeoplesearch.com/address/129-S...
1,Jose,Rosa,Munoz,109 S Palomas Av,109,S,Palomas,Av,None,None,https://www.fastpeoplesearch.com/address/109-S...
2,Richard,NaN,Ortiz,15 S Cuesta Av,15,S,Cuesta,Av,None,None,https://www.fastpeoplesearch.com/address/15-S-...
3,Elizabeth,NaN,Newton,130 S Westmoreland Av,130,S,Westmoreland,Av,None,None,https://www.fastpeoplesearch.com/address/130-S...
4,Christina,NaN,Fraijo,142 S Westmoreland Av,142,S,Westmoreland,Av,None,None,https://www.fastpeoplesearch.com/address/142-S...
...,...,...,...,...,...,...,...,...,...,...,...
1183,Travis,NaN,Woodard,231 W Estrella Av,231,W,Estrella,Av,None,None,https://www.fastpeoplesearch.com/address/231-W...
1184,Steven,Kathy,Turnsen,201 W Rocalla Av,201,W,Rocalla,Av,None,None,https://www.fastpeoplesearch.com/address/201-W...
1185,Gregory,NaN,Walker,521 W Rocalla Av,521,W,Rocalla,Av,None,None,https://www.fastpeoplesearch.com/address/521-W...
1186,Cecilia,NaN,Montes,725 W Solana Av,725,W,Solana,Av,None,None,https://www.fastpeoplesearch.com/address/725-W...


# RUN

In [ ]:
# Loop through each address
count = 0
discord = 'https://discord.com/api/webhooks/1169906230316240937/0EXUlGJRLPb1Tclm-cWNTeWmd-mCiNc7t80gtMTIY3a3DY-eGpje8BKxuu4m1TANQiNT'
discord2 = 'https://discord.com/api/webhooks/1169922010848174132/pE5O41sBx1Hqjbs3iis3aQdbwIJG57TFt6vv6NBNWtNk8lXdwA72o_4cHk1WgdwWD41P'

for i in final_df['addy_link']:

    print(count)
    current_name = final_df['first'][count] + '-' + final_df['last'][count]
    print('searching for ' + current_name)

    # Open Chrome
    chromedriver_path = "C:\\Users\\kin\\Desktop\\chromedriver.exe"
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service)
    driver.get(i)
    
    print('link: ' + i)
    # Sleep if denied
    if "Access Denied" in driver.page_source:
        print("Access Denied")
        time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Grab list of names on page
    # people_list = soup.find_all('div', {'class': 'card'})

    # Loop through all names on page 
    # for people in people_list:
    hrefs = [a['href'] for a in soup.find_all('a', href=True)]
        
        # Find people's names 
        # person_name = people.find('span', class_ = 'larger').text
    
        # Check if first names = addy on CSV
        # if final_df['first'][count] in person_name:
    
            # List all hrefs
            # hrefs = [a['href'] for a in soup.find_all('a', href=True)]
    
    # Find ID for Card
    for j in hrefs:
        if current_name.lower() in j:
            print('found ' + current_name)
            if j.startswith('https'):
                print('ID was a link!')
                id = (j.split('/', 3)[-1])
                print("id: " + id)
                numbers = find_phone_num(id)
                if numbers == "no num found":
                    print('couldn''t find anything')
                    phones.append("NA")
                    break
                phones.append(numbers)
                print(numbers)
                data2 = {
                    'content': numbers
                }
                x = requests.post(discord2, json=data2)
                script = "Hello " + final_df['first'][count] + ",\n\nMy name is Graeme, and I'm a local investor in Tucson. I came across your property at " + final_df['addy'][count] + ", and wanted to personally reach out. I'm interested in purchasing it, regardless of its condition or weather it has tenants. " + "\n\n" + "If you're open to selling or have any questions, please feel free to text back or give me a call :). " + "\n\n" + "Warm regards," + "\n\n" + "Graeme"
                print(script)
                data = {
                    'content': script
                }
                x = requests.post(discord, json=data)
                break
            else:
                print('ID was not a link!')
                id = j
                numbers = find_phone_num(id)
                if numbers == "no num found":
                    print('couldn''t find anything')
                    phones.append("NA")
                    break
                phones.append(numbers)
                print(numbers)
                data2 = {
                    'content': numbers
                }
                x = requests.post(discord2, json=data2)
                script = "Hello " + final_df['first'][count] + ",\n\nMy name is Graeme, and I'm a local investor in Tucson. I came across your property at " + final_df['addy'][count] + ", and wanted to personally reach out. I'm interested in purchasing it, regardless of its condition or weather it has tenants. " + "\n\n" + "If you're open to selling or have any questions, please feel free to text back or give me a call :). " + "\n\n" + "Warm regards," + "\n\n" + "Graeme"
                print(script)
                data = {
                    'content': script
                }
                x = requests.post(discord, json=data)
                break
            # Find phone number from ID
            
    else:
        print('couldn''t find anything')
        phones.append("NA")
        
    count = count + 1
    driver.quit()
    time.sleep(1)  


0
searching for John-Arredondo
link: https://www.fastpeoplesearch.com/address/129-S-Melrose-Av_tucson-az
couldnt find anything
1
searching for Jose-Munoz
link: https://www.fastpeoplesearch.com/address/109-S-Palomas-Av_tucson-az
found Jose-Munoz
ID was not a link!
(520) 622-6725
Hello Jose,

My name is Graeme, and I'm a local investor in Tucson. I came across your property at 109 S Palomas Av, and wanted to personally reach out. I'm interested in purchasing it, regardless of its condition or weather it has tenants. 

If you're open to selling or have any questions, please feel free to text back or give me a call :). 

Warm regards,

Graeme
2
searching for Richard-Ortiz
link: https://www.fastpeoplesearch.com/address/15-S-Cuesta-Av_tucson-az
found Richard-Ortiz
ID was a link!
id: richard-ortiz_id_G-2368906110521954173
(520) 623-9798
Hello Richard,

My name is Graeme, and I'm a local investor in Tucson. I came across your property at 15 S Cuesta Av, and wanted to personally reach out. I'm 